<a href="https://colab.research.google.com/github/hannahbhchou/song-recommendation/blob/main/Lyrics_Scrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
GENIUS_API_TOKEN='zTJcf2nWD9L_uvBVw8MaGLF8gY1prbNKlNiKfhJdey-_ybEkXzBuX1lOTsHaMnSi'

In [ ]:
import requests #Scrape data from an HTML document
from bs4 import BeautifulSoup # I/O
import os #Search and manipulate strings
import re
import pandas as pd
import time
import csv
import random

In [ ]:
artist_url = "https://gist.githubusercontent.com/hannahbhchou/096fc3192c18436046f490fc195fa265/raw/e230da3dfc1f42d32078b00525167283deed8010/best_selling_artist.csv"

best_selling_artist_df = pd.read_csv(artist_url)

In [ ]:
best_selling_artist_df.head()

,Artist,Country / Market,Claimed sales
0,The Beatles,United Kingdom,600 million[40][41]500 million[42]
1,Elvis Presley,United States,600 million[52][53]500 million[54]
2,Michael Jackson,United States,350 million[66][67]300 million[68][69]
3,Elton John,United Kingdom,300 million[74][75]250 million[76][77]
4,Madonna,United States,300 million[86][87]275 million[88]


In [ ]:
best_selling_artist_df.iloc[75, 0] = "P!nk"

In [ ]:
broken_artist = ["Celine Dion", "Guns N Roses", "The Carpenters", "R.E.M.", "The Black Eyed Peas"]

In [ ]:
index_names = []

for i in range(0, len(broken_artist)):
  index_names.append(best_selling_artist_df[best_selling_artist_df['Artist'] == broken_artist[i]].index)

for index in index_names:
  best_selling_artist_df.drop(index, inplace = True)

In [ ]:
# Get artist object from Genius API
def request_artist_info(artist_name, page):
    base_url = 'https://api.genius.com'
    headers = {'Authorization': 'Bearer ' + GENIUS_API_TOKEN}
    search_url = base_url + '/search?per_page=10&page=' + str(page)
    data = {'q': artist_name}
    response = requests.get(search_url, data=data, headers=headers, timeout = 10)
    return response# Get Genius.com song url's from artist object
    
def request_song_url(artist_name, song_cap):
    page = 1
    songs = []
    
    while True:
        response = request_artist_info(artist_name, page)
        json = response.json()        # Collect up to song_cap song objects from artist
        song_info = []
        for hit in json['response']['hits']:
            if artist_name.lower() in hit['result']['primary_artist']['name'].lower():
                song_info.append(hit)
    
        # Collect song URL's from song objects
        for song in song_info:
            if (len(songs) < song_cap):
                url = song['result']['url']
                songs.append(url)
            
        if (len(songs) == song_cap):
            break
        else:
            page += 1
        
    print('Found {} songs by {}'.format(len(songs), artist_name))
    return songs

In [ ]:
def request_10_song_url(artist_name):
    page = 1
    songs = []
    song_cap = 10
    
    while True:
        response = request_artist_info(artist_name, page)
        json = response.json()        # Collect up to song_cap song objects from artist
        song_info = []
        for hit in json['response']['hits']:
            if artist_name.lower() in hit['result']['primary_artist']['name'].lower():
                song_info.append(hit)
    
        # Collect song URL's from song objects
        for song in song_info:
            if (len(songs) < song_cap):
                url = song['result']['url']
                songs.append(url)
            
        if (len(songs) == song_cap):
            break
        else:
            page += 1
        
    return songs

In [ ]:
randomlist = []
for i in range(0,1050):
  n = random.randint(1,5)
  randomlist.append(n)

In [ ]:
sleep_sequence = 0

with open('artist_url.csv','w') as f1:
    writer=csv.writer(f1, delimiter=',',lineterminator='\n',)
    for artist in best_selling_artist_df["Artist"]:
      song_urls = request_10_song_url(artist)
      for song_url in song_urls:
        writer.writerow([song_url])
      time.sleep(randomlist[sleep_sequence])
      sleep_sequence += 1

In [ ]:
def find_tag(tag):
    if tag.name != "div":
        return False

    cls = tag.attrs.get("class", [])

    return any(
        c.startswith(w) for c in cls for w in ("lyrics", "SongPage__Section")
    )


def scrape_song_lyrics(url):
    page = requests.get(url)
    html = BeautifulSoup(page.text, "html.parser")
    lyrics = html.find(find_tag).get_text(separator = "\n")
    # remove identifiers like chorus, verse, etc
    lyrics = re.sub(r'[\(\[].*?[\)\]]', '', lyrics)
    # remove empty lines
    lyrics = os.linesep.join([s for s in lyrics.splitlines() if s]) 
    return lyrics

In [ ]:
path = '/content/drive/Shareddrives/MBD Term 3 Group 3/Natural Language Processing/Lyrics/song_urls.csv'
song_df = pd.read_csv(path)

In [ ]:
sleep_sequence = 0

with open('song_lyrics.csv','w') as f1:
    writer=csv.writer(f1, delimiter=',',lineterminator='\n',)
    for url in song_df["URL"]:
      lyrics = scrape_song_lyrics(url)
      writer.writerow([url, lyrics])
      time.sleep(randomlist[sleep_sequence])
      sleep_sequence += 1